<a href="https://colab.research.google.com/github/emily2925/machine_learning_algorithm/blob/main/Clustering%E5%AD%B8%E7%BF%92%E7%AD%86%E8%A8%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



*   此筆記整理Kmeans,Hierarchy,Kmodes之原理、限制、實作程式和結果解釋
*   Kmeans原理：https://www.youtube.com/watch?v=4b5d3muPQmA&ab_channel=StatQuestwithJoshStarmer
*   Hierarchy原理：https://www.youtube.com/watch?v=7xHsRkOdVwo&ab_channel=StatQuestwithJoshStarmer
*   KModes參考程式：https://www.analyticsvidhya.com/blog/2021/06/kmodes-clustering-algorithm-for-categorical-data/
\






**Kmeans**

**1.原理**
先選定k個點

In [ ]:
# 